In [44]:
from sklearn.model_selection import KFold
import numpy as np
import sys
sys.path.append('/home/ec2-user/SageMaker/mariano/notebooks/07. Simulation/')
from utils.data_item import DataItem
alldata_file = '/home/ec2-user/SageMaker/mariano/data/simulations/data/labeled_data_latest_08072022.csv'

data = np.array(open(alldata_file, 'r').read().splitlines()[1:])
print(data[:5])
np.random.shuffle(data)
print(data[:5])

folds = {
    'split_1': {},
    'split_2': {},
    'split_3': {},
    'split_4': {},
    'split_5': {}
}


kf = KFold(n_splits=5)
i=1
for train_args, test_args in list(kf.split(data)):
    folds[f'split_{i}']['train']=data[train_args]
    folds[f'split_{i}']['test']=data[test_args]
    i+=1
    
def has_repeated_values(list_a,list_b):
    set_a = set(list_a)
    set_b = set(list_b)
    return len(set_a.intersection(set_b))!=0

data_dirpath = '/home/ec2-user/SageMaker/mariano/data/simulations/data/'
i=1
for split in folds:
    train = folds[split]['train']
    test = folds[split]['test']
    
    train_relevant_count=len([elem for elem in train if elem.endswith('R')])
    train_irrelevant_count=len([elem for elem in train if elem.endswith('I')])
    assert train_relevant_count+train_irrelevant_count==len(train)
    
    
    test_relevant_count=len([elem for elem in test if elem.endswith('R')])
    test_irrelevant_count=len([elem for elem in test if elem.endswith('I')])
    assert test_relevant_count+test_irrelevant_count==len(test)
    
    print(f'split #{i} - train: {train_relevant_count:5}/{len(train):5} - test: {test_relevant_count:5}/{len(test):5}')
    
    assert set(train).union(set(test)) == (set(data))
    assert not has_repeated_values(train,test)
    
    writer = open(data_dirpath+f'split_{i}/train.csv', 'w')
    writer.write('id;label\n')
    writer.write('\n'.join(train))
    writer.close()
    
    writer = open(data_dirpath+f'split_{i}/test.csv', 'w')
    writer.write('id;label\n')
    writer.write('\n'.join(test))
    writer.close()
    i+=1

['1293118416;R' '1291398730;R' '1325747973;R' '1287771491;R'
 '1291249197;R']
['1287357804;I' '1291246656;R' '1287275127;I' '1270384613;I'
 '1287637549;I']
split #1 - train:   519/ 5825 - test:   135/ 1457
split #2 - train:   533/ 5825 - test:   121/ 1457
split #3 - train:   489/ 5826 - test:   165/ 1456
split #4 - train:   536/ 5826 - test:   118/ 1456
split #5 - train:   539/ 5826 - test:   115/ 1456


In [37]:
len(set(train).union(set(test)))

7282

In [19]:
folds['split_1']['train']

array(['1313946262;I', '1316348093;I', '1291408681;R', '1324228783;I',
       '1282718684;I', '1270525654;I', '1288866068;I', '1325811762;I',
       '1291192600;R', '1284879905;I', '1284842058;I', '1270332231;I',
       '1282799254;I', '1287326073;I', '1325729475;R', '1291164296;I'],
      dtype='<U12')

In [13]:
from sklearn.model_selection import StratifiedKFold, KFold
import numpy as np
all_arguments = list(range(20))
kf = KFold(n_splits=5)

for train_args, test_args in list(kf.split(all_arguments)):
    print(train_args)
    print(test_args)
    print()

[ 4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
[0 1 2 3]

[ 0  1  2  3  8  9 10 11 12 13 14 15 16 17 18 19]
[4 5 6 7]

[ 0  1  2  3  4  5  6  7 12 13 14 15 16 17 18 19]
[ 8  9 10 11]

[ 0  1  2  3  4  5  6  7  8  9 10 11 16 17 18 19]
[12 13 14 15]

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
[16 17 18 19]



In [5]:
import numpy as np
import os

random_generator = np.random.default_rng(2022)

testing_ratio = 0.2
training_ratio = 0.8
dataset_size = len(data)
training_size = int(dataset_size*training_ratio)
testing_size = dataset_size - training_size
print(f'dataset size:  {dataset_size:6,}')
print(f'training size: {training_size:6,}')
print(f'testing size:  {testing_size:6,}')
assert testing_size+training_size==dataset_size
assert testing_ratio+training_ratio==1

data_dirpath = '/home/ec2-user/SageMaker/mariano/data/simulations/data/'
assert os.path.exists(data_dirpath)

no_of_splits=5

for i in range(1,1+no_of_splits):
    training_args = random_generator.choice(range(dataset_size), size=training_size,replace=False)
    testing_args  = [arg for arg in range(dataset_size) if not arg in training_args]
    assert len(set(training_args).intersection(set(testing_args)))==0
    assert len(set(training_args).union(set(testing_args)))==dataset_size
    assert len(set(training_args))==len(training_args)
    assert len(set(testing_args))==len(testing_args)
    
    training_data = [data[arg] for arg in training_args]
    testing_data  = [data[arg] for arg in testing_args]
    
    assert len(set(training_data).intersection(testing_data))==0
    assert len(set(training_data).union(testing_data))==dataset_size
    
    if not os.path.exists(data_dirpath+f'split_{i}/'):
        os.mkdir(data_dirpath+f'split_{i}/')
        
    training_writer = open(data_dirpath+f'split_{i}/train.csv', 'w')
    training_writer.write('id;label\n')
    for elem in training_data:
        training_writer.write(elem+'\n')
    
    testing_writer = open(data_dirpath+f'split_{i}/test.csv', 'w')
    testing_writer.write('id;label\n')
    for elem in testing_data:
        testing_writer.write(elem+'\n')
    
    

dataset size:   7,282
training size:  5,825
testing size:   1,457


In [12]:
np.random.choice?